#Earth Surface Temperature Analytics

This noteboook contains the code and results involving an investigation where we answer questions and make predictions based on the datasets GlobalLandTemperaturesByCountry.csv and GlobalLandTemperaturesByCity.csv obtained from Climate Change: Earth Surface Temperature Data created by the Berkeley Earth Surface Temperature Study, i.e. https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data in December 2020. As the investigation of these datasets are based on First World Countries we also use the dataset FirstWorldCountries2020.csv obtained from https://worldpopulationreview.com/country-rankings/first-world-countries in January 2021. Due to the large scale of the data used in this project, i.e. Multicore Big Data processing, we can use SparkSQL and/or DataFrame API (PySpark) from Apache Spark to create, transform and perform analytics on Spark DataFrames, which is executed via a Hadoop cluster on Databricks. Please ensure that the datasets used in this project are imported/stored to the Databricks File System, DBFS. 

The goals of this project which are covered in this notebook are:

1.   Investigate which First World Countries (top five) have had the biggest change in average land temperature between 1850 and 2013.
2.   Investigate which Cities (top five) of the five First World Countries from Goal 1 have seen the biggest change in average land temperature between 1850 and 2013.
3.   Predict the average land temperature of the City found to have the greatest change in average land temperature from Goal 2 for the years 2014-2020.

The specific timeframe of the years 1850 to 2013 is selected due to the increased industrialisation throughout the world since the mid-20th century.

In order to achieve the goals stated above, a data analytic pipeline is used. The pipeline for this project comprises of the following stages:


1.   Data Collection
2.   Data Transformation
3.   Feature Engineering (Goal 3 only)
4.   Algorithm Selection (Goal 3 only, for Predictive Analysis), i.e. Supervised Machine Learning Algorithms Linear Regression, Decision Tree Regression and Random Forest Regression
5.   Training (Goal 3 only, for Predictive Analysis)
6.   Testing (Goal 3 only, for Predictive Analysis)
7.   Descriptive or Predictive Analysis
8.   Data Visualisation

Due to the nature of the goals, i.e. Goal 2 depends on the results of Goal 1 and Goal 3 depends on the results of Goal 2, we shall implement the data analytic pipeline three times, once for each goal.

NOTE: The features AverageTemperature and AverageTemperatureUncertainty are measured in degree Celcius (°C).

##Goal 1:  Top five First World Countries that have had the biggest change in average land temperature between 1850 and 2013

In this section we aim to discover, using the Global Land Temperatures By Country dataset and the First World Countries 2020 dataset, the top five First World Countries that have had the biggest change in average land temperature between 1850 and 2013. To achieve this aim, we will use the data analytic pipeline described at the beginning of this notebook.

###Stage 1: Data Collection

Read in the Global Land Temeratures By Country dataset and First World Countries 2020 dataset as Spark DataFrames.

In [0]:
# Read Global Land Temeratures By Country dataset from  /FileStore/tables/GlobalLandTemperaturesByCountry.csv
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, TimestampType

country_land_temp_sdf = (sqlContext.read.format("csv").
  option("header", "true").
  option("delimiter", ",").                      
  option("nullValue", "NA").                      
  option("inferSchema", True).
  load("/FileStore/tables/GlobalLandTemperaturesByCountry.csv"))

country_land_temp_sdf.printSchema()

root
-- dt: timestamp (nullable = true)
-- AverageTemperature: double (nullable = true)
-- AverageTemperatureUncertainty: double (nullable = true)
-- Country: string (nullable = true)

In [0]:
# See Global Land Temeratures By Country data
display(country_land_temp_sdf)

dt,AverageTemperature,AverageTemperatureUncertainty,Country
1743-11-01T00:00:00.000+0000,4.3839999999999995,2.294,Åland
1743-12-01T00:00:00.000+0000,null,null,Åland
1744-01-01T00:00:00.000+0000,null,null,Åland
1744-02-01T00:00:00.000+0000,null,null,Åland
1744-03-01T00:00:00.000+0000,null,null,Åland
1744-04-01T00:00:00.000+0000,1.53,4.68,Åland
1744-05-01T00:00:00.000+0000,6.702000000000001,1.789,Åland
1744-06-01T00:00:00.000+0000,11.609000000000002,1.577,Åland
1744-07-01T00:00:00.000+0000,15.342,1.41,Åland
1744-08-01T00:00:00.000+0000,null,null,Åland


In [0]:
# Read First World Countries 2020 dataset from  /FileStore/tables/FirstWorldCountries2020.csv
first_world_countries_sdf = (sqlContext.read.format("csv").
  option("header", "true").
  option("delimiter", ",").                      
  option("nullValue", "NA").                      
  option("inferSchema", True).
  load("/FileStore/tables/FirstWorldCountries2020.csv"))

first_world_countries_sdf.printSchema()

root
-- country: string (nullable = true)
-- humanDevelopmentIndex: double (nullable = true)
-- pop2020: double (nullable = true)

In [0]:
# See First World Countries 2020 data
display(first_world_countries_sdf)

country,humanDevelopmentIndex,pop2020
Norway,0.953,5421.241
Switzerland,0.944,8654.622
Australia,0.939,25499.884
Ireland,0.938,4937.786
Germany,0.936,83783.942
Iceland,0.935,341.243
Hong Kong,0.933,7496.981
Sweden,0.933,10099.265
Singapore,0.932,5850.342
Netherlands,0.931,17134.872


###Stage 2: Data Transformation

Taking country_land_temp_sdf, extract year and month from dt and select the required features.

In [0]:
# Extract year and month from dt and select features

from pyspark.sql.functions import month, year

country_land_temp_sdf_cleaned = country_land_temp_sdf.select(year(country_land_temp_sdf['dt']).alias('year'),
                                                             month(country_land_temp_sdf['dt']).alias('month'),
                                                             country_land_temp_sdf['AverageTemperature'],
                                                             country_land_temp_sdf['Country'])

country_land_temp_sdf_cleaned.show()

+----+-----+-------------------+-------+
year|month| AverageTemperature|Country|
+----+-----+-------------------+-------+
1743| 11| 4.3839999999999995| Åland|
1743| 12| null| Åland|
1744| 1| null| Åland|
1744| 2| null| Åland|
1744| 3| null| Åland|
1744| 4| 1.53| Åland|
1744| 5| 6.702000000000001| Åland|
1744| 6| 11.609000000000002| Åland|
1744| 7| 15.342| Åland|
1744| 8| null| Åland|
1744| 9| 11.702| Åland|
1744| 10| 5.477| Åland|
1744| 11| 3.407| Åland|
1744| 12| -2.181| Åland|
1745| 1| -3.85| Åland|
1745| 2| -6.574999999999998| Åland|
1745| 3| -4.195| Åland|
1745| 4|-0.9660000000000002| Åland|
1745| 5| null| Åland|
1745| 6| null| Åland|
+----+-----+-------------------+-------+
only showing top 20 rows

In [0]:
# Check that year and month are of data type int
country_land_temp_sdf_cleaned.dtypes

Out[6]: [('year', 'int'),
 ('month', 'int'),
 ('AverageTemperature', 'double'),
 ('Country', 'string')]

We want years 1850-2013.
Filter country_land_temp_sdf_cleaned for years greater than 1849.

In [0]:
# Filter year > 1849
country_land_temp_sdf_cleaned = country_land_temp_sdf_cleaned.filter(country_land_temp_sdf_cleaned['year'] > 1849)[['year', 'month', 'AverageTemperature', 'Country']]

country_land_temp_sdf_cleaned.show()

+----+-----+-------------------+-------+
year|month| AverageTemperature|Country|
+----+-----+-------------------+-------+
1850| 1| -9.082999999999998| Åland|
1850| 2| -2.309| Åland|
1850| 3| -4.801| Åland|
1850| 4| 1.242| Åland|
1850| 5| 7.92| Åland|
1850| 6| 12.704| Åland|
1850| 7| 15.905999999999999| Åland|
1850| 8| 16.317999999999998| Åland|
1850| 9| 10.483999999999998| Åland|
1850| 10| 5.218999999999999| Åland|
1850| 11| 0.6859999999999999| Åland|
1850| 12| 1.4980000000000002| Åland|
1851| 1|-1.4399999999999995| Åland|
1851| 2| -3.052| Åland|
1851| 3|-4.5280000000000005| Åland|
1851| 4| 2.117| Åland|
1851| 5| 4.945| Åland|
1851| 6| 11.352| Åland|
1851| 7| 14.741| Åland|
1851| 8| 14.122| Åland|
+----+-----+-------------------+-------+
only showing top 20 rows

Drop any null values from country_land_temp_sdf_cleaned.

In [0]:
# Drop nulls
country_land_temp_sdf_cleaned = country_land_temp_sdf_cleaned.dropna()

country_land_temp_sdf_cleaned.createOrReplaceTempView("country_land_temp_sdf_cleaned")

country_land_temp_sdf_cleaned.show()

+----+-----+-------------------+-------+
year|month| AverageTemperature|Country|
+----+-----+-------------------+-------+
1850| 1| -9.082999999999998| Åland|
1850| 2| -2.309| Åland|
1850| 3| -4.801| Åland|
1850| 4| 1.242| Åland|
1850| 5| 7.92| Åland|
1850| 6| 12.704| Åland|
1850| 7| 15.905999999999999| Åland|
1850| 8| 16.317999999999998| Åland|
1850| 9| 10.483999999999998| Åland|
1850| 10| 5.218999999999999| Åland|
1850| 11| 0.6859999999999999| Åland|
1850| 12| 1.4980000000000002| Åland|
1851| 1|-1.4399999999999995| Åland|
1851| 2| -3.052| Åland|
1851| 3|-4.5280000000000005| Åland|
1851| 4| 2.117| Åland|
1851| 5| 4.945| Åland|
1851| 6| 11.352| Åland|
1851| 7| 14.741| Åland|
1851| 8| 14.122| Åland|
+----+-----+-------------------+-------+
only showing top 20 rows

Now we take first_world_countries_sdf. In order to get the first world countries from this dataframe we will filter the countries where the humanDevelopmentIndex is greater than the average.

First calculate the average of humanDevelopmentIndex in first_world_countries_sdf.

In [0]:
# Calculate average of humanDevelopmentIndex

from pyspark.sql.functions import avg, col

average_hdi = first_world_countries_sdf.select(avg(col('humanDevelopmentIndex')).alias('avgHumanDevelopmentIndex')).collect()
average_hdi[0]

Out[9]: Row(avgHumanDevelopmentIndex=0.7099139784946235)

Then from first_world_countries_sdf, filter rows where the feature humanDevelopmentIndex is greater than avgHumanDevelopmentIndex and select the feature country, renamed as FirstWorldCountry.

In [0]:
# Filter humanDevelopmentIndex > avgHumanDevelopmentIndex and select FirstWorldCountry
first_world_countries_sdf_cleaned = first_world_countries_sdf.filter(first_world_countries_sdf['humanDevelopmentIndex'] > average_hdi[0]['avgHumanDevelopmentIndex']).\
                                                            select(first_world_countries_sdf['country'].alias('FirstWorldCountry'))

first_world_countries_sdf_cleaned.createOrReplaceTempView("first_world_countries_sdf_cleaned")

first_world_countries_sdf_cleaned.show()

+-----------------+
FirstWorldCountry|
+-----------------+
 Norway|
 Switzerland|
 Australia|
 Ireland|
 Germany|
 Iceland|
 Hong Kong|
 Sweden|
 Singapore|
 Netherlands|
 Denmark|
 Canada|
 United States|
 United Kingdom|
 Finland|
 New Zealand|
 Liechtenstein|
 Belgium|
 Japan|
 Austria|
+-----------------+
only showing top 20 rows

Now merge country_land_temp_sdf_cleaned with first_world_countries_sdf_cleaned on Country=FirstWorldCountry, projecting year, month, AverageTemperature and Country in ascending order.

In [0]:
# Merge
fw_country_land_temp_sdf = sqlContext.sql("select year, month, AverageTemperature, Country FROM country_land_temp_sdf_cleaned join first_world_countries_sdf_cleaned ON Country=FirstWorldCountry ORDER BY Country, year, month")

fw_country_land_temp_sdf.show()

+----+-----+--------------------+-------+
year|month| AverageTemperature|Country|
+----+-----+--------------------+-------+
1850| 1|-0.16000000000000014|Albania|
1850| 2| 4.27|Albania|
1850| 3| 4.868|Albania|
1850| 4| 10.7|Albania|
1850| 5| 14.758|Albania|
1850| 6| 20.137999999999998|Albania|
1850| 7| 21.84|Albania|
1850| 8| 22.736|Albania|
1850| 9| 16.512|Albania|
1850| 10| 12.763|Albania|
1850| 11| 7.9479999999999995|Albania|
1850| 12| 4.4430000000000005|Albania|
1851| 1| 3.242|Albania|
1851| 2| 4.482|Albania|
1851| 3| 7.609|Albania|
1851| 4| 12.173|Albania|
1851| 5| 15.272|Albania|
1851| 6| 19.055999999999997|Albania|
1851| 7| 21.946|Albania|
1851| 8| 21.638|Albania|
+----+-----+--------------------+-------+
only showing top 20 rows

Check if any Country in country_land_temp_sdf_cleaned did not match with a FirstWorldCountry in first_world_countries_sdf_cleaned in the merge operation.

In [0]:
# Number of missing countries

from pyspark.sql.functions import countDistinct

# Number of countries in first_world_countries_sdf_cleaned
num_first_world_countries = first_world_countries_sdf_cleaned.select(countDistinct('FirstWorldCountry').alias('num_of_FWC')).collect()
# Number of countries in fw_country_land_temp_sdf
num_countries = fw_country_land_temp_sdf.select(countDistinct('Country').alias('num_of_countries')).collect()

# Calculate the difference
num_missing_countries = num_first_world_countries[0]['num_of_FWC'] - num_countries[0]['num_of_countries']

# Print
missing_countries = "%d countries from first_world_countries_sdf_cleaned are missing in fw_country_land_temp_sdf after the merge operation" %(num_missing_countries)
print(missing_countries)

7 countries from first_world_countries_sdf_cleaned are missing in fw_country_land_temp_sdf after the merge operation

There are 7 countries in the First World Countries 2020 dataset that are not in the Global Land Temperatures By Country Dataset. What are they and why are they missing?

Do another merge operation on the same DataFrames, but this time do a right join and remove the inner part.

In [0]:
# Right merge
missing_country_sdf = sqlContext.sql("select DISTINCT Country, FirstWorldCountry FROM country_land_temp_sdf_cleaned right join first_world_countries_sdf_cleaned ON Country=FirstWorldCountry WHERE Country IS NULL")

missing_country_sdf.show()

+-------+--------------------+
Country| FirstWorldCountry|
+-------+--------------------+
 null| Maldives|
 null|Saint Kitts and N...|
 null|Saint Vincent and...|
 null| Trinidad and Tobago|
 null| Brunei|
 null| Antigua and Barbuda|
 null|Bosnia and Herzeg...|
+-------+--------------------+

A possible reason for the absence of records for the countries above in the Global Land Temperatures By Country dataset would be due to the fact that these countries were not founded at the time of records being kept for this dataset. Taking for example the Maldives, which was not founded until 1965 (https://www.britannica.com/place/Maldives/History).

###Stage 3: Descriptive Analysis

Taking fw_country_land_temp_sdf, calculate the yearly average temperature for each First World Country and project to a new DataFrame fw_country_yearly_land_temp_sdf with features year, Country and YearlyAverageTemperature.

In [0]:
# YearlyAverageTemperature for each First World Country
fw_country_yearly_land_temp_sdf = fw_country_land_temp_sdf.groupBy('year', 'Country').agg(avg(col('AverageTemperature')).alias('YearlyAverageTemperature')).orderBy('Country', 'year')

fw_country_yearly_land_temp_sdf.show()

+----+-------+------------------------+
year|Country|YearlyAverageTemperature|
+----+-------+------------------------+
1850|Albania| 11.734666666666667|
1851|Albania| 12.3155|
1852|Albania| 12.744583333333333|
1853|Albania| 12.811166666666665|
1854|Albania| 11.951666666666668|
1855|Albania| 12.333583333333335|
1856|Albania| 12.06075|
1857|Albania| 11.994666666666665|
1858|Albania| 11.505416666666667|
1859|Albania| 12.675833333333332|
1860|Albania| 12.693916666666668|
1861|Albania| 12.510083333333332|
1862|Albania| 12.938666666666665|
1863|Albania| 13.166083333333335|
1864|Albania| 11.73975|
1865|Albania| 12.415666666666668|
1866|Albania| 12.802333333333332|
1867|Albania| 12.665583333333332|
1868|Albania| 12.73975|
1869|Albania| 12.534833333333333|
+----+-------+------------------------+
only showing top 20 rows

Taking fw_country_yearly_land_temp_sdf, calculate the max and min average temperature for each First World Country and project each computation to new DataFrames max_fw_country_land_temp_sdf and min_fw_country_land_temp_sdf. Then merge these new DataFrames and call this maxmin_fw_country_land_temp_sdf.

In [0]:
# max_fw_country_land_temp_sdf

from pyspark.sql.functions import min, max

max_fw_country_land_temp_sdf = fw_country_yearly_land_temp_sdf.groupBy('Country').agg(max(col('YearlyAverageTemperature')).alias('MaxAverageTemperature')).orderBy('Country')

max_fw_country_land_temp_sdf.createOrReplaceTempView("max_fw_country_land_temp_sdf")

max_fw_country_land_temp_sdf.show()

+----------+---------------------+
 Country|MaxAverageTemperature|
+----------+---------------------+
 Albania| 14.993875000000001|
 Algeria| 25.21566666666666|
 Andorra| 12.994416666666668|
 Argentina| 15.688999999999998|
 Armenia| 11.509500000000003|
 Australia| 22.896833333333333|
 Austria| 8.164916666666667|
Azerbaijan| 14.173875|
 Bahamas| 26.2295|
 Bahrain| 28.069666666666667|
 Barbados| 27.84466666666667|
 Belarus| 8.2665|
 Belgium| 11.305333333333335|
 Botswana| 23.395166666666665|
 Brazil| 25.958750000000006|
 Bulgaria| 13.411874999999998|
 Canada| -1.6406666666666676|
 Chile| 10.336999999999998|
 China| 9.295|
 Colombia| 25.879166666666663|
+----------+---------------------+
only showing top 20 rows

In [0]:
# min_fw_country_land_temp_sdf

min_fw_country_land_temp_sdf = fw_country_yearly_land_temp_sdf.groupBy('Country').agg(min(col('YearlyAverageTemperature')).alias('MinAverageTemperature')).orderBy('Country').\
                                select(col('Country').alias('Country_x'), 'MinAverageTemperature')

min_fw_country_land_temp_sdf.createOrReplaceTempView("min_fw_country_land_temp_sdf")

min_fw_country_land_temp_sdf.show()

+----------+---------------------+
 Country_x|MinAverageTemperature|
+----------+---------------------+
 Albania| 11.505416666666667|
 Algeria| 21.91808333333333|
 Andorra| 9.934583333333334|
 Argentina| 11.729249999999999|
 Armenia| 6.994583333333334|
 Australia| 20.1885|
 Austria| 4.609333333333333|
Azerbaijan| 9.485499999999998|
 Bahamas| 24.2045|
 Bahrain| 24.540666666666667|
 Barbados| 25.518333333333334|
 Belarus| 3.6961666666666653|
 Belgium| 7.801583333333334|
 Botswana| 20.40975|
 Brazil| 23.7975|
 Bulgaria| 8.955583333333331|
 Canada| -6.8946666666666685|
 Chile| 7.070125|
 China| 5.448083333333333|
 Colombia| 23.784583333333334|
+----------+---------------------+
only showing top 20 rows

In [0]:
# Merge
maxmin_fw_country_land_temp_sdf = sqlContext.sql("select Country, maxAverageTemperature, minAverageTemperature FROM max_fw_country_land_temp_sdf join min_fw_country_land_temp_sdf ON Country=Country_x ORDER BY Country")

maxmin_fw_country_land_temp_sdf.show()

+----------+---------------------+---------------------+
 Country|maxAverageTemperature|minAverageTemperature|
+----------+---------------------+---------------------+
 Albania| 14.993875000000001| 11.505416666666667|
 Algeria| 25.21566666666666| 21.91808333333333|
 Andorra| 12.994416666666668| 9.934583333333334|
 Argentina| 15.688999999999998| 11.729249999999999|
 Armenia| 11.509500000000003| 6.994583333333334|
 Australia| 22.896833333333333| 20.1885|
 Austria| 8.164916666666667| 4.609333333333333|
Azerbaijan| 14.173875| 9.485499999999998|
 Bahamas| 26.2295| 24.2045|
 Bahrain| 28.069666666666667| 24.540666666666667|
 Barbados| 27.84466666666667| 25.518333333333334|
 Belarus| 8.2665| 3.6961666666666653|
 Belgium| 11.305333333333335| 7.801583333333334|
 Botswana| 23.395166666666665| 20.40975|
 Brazil| 25.958750000000006| 23.7975|
 Bulgaria| 13.411874999999998| 8.955583333333331|
 Canada| -1.6406666666666676| -6.8946666666666685|
 Chile| 10.336999999999998| 7.070125|
 China| 9.295| 5.448083333333333|
 Colombia| 25.879166666666663| 23.784583333333334|
+----------+---------------------+---------------------+
only showing top 20 rows

Taking maxmin_fw_country_land_temp_sdf, calculate the difference of max and min average temperature for each First World Country and project to a new DataFrame diff_fw_country_land_temp_sdf. We use the absolute value function to ensure the values are positive.

Additionally, as we want to see the First World Countries that have had the biggest change in average land temperature between 1850 and 2013, we order the feature diffAverageTemperature in descending order.

In [0]:
# diff_fw_country_land_temp_sdf

from pyspark.sql.functions import abs

diff_fw_country_land_temp_sdf = maxmin_fw_country_land_temp_sdf.select('Country',
                                                                       abs(col('maxAverageTemperature') - col('minAverageTemperature')).alias('diffAverageTemperature')).orderBy('diffAverageTemperature', ascending=False)

diff_fw_country_land_temp_sdf.show()

+--------------------+----------------------+
 Country|diffAverageTemperature|
+--------------------+----------------------+
 Saudi Arabia| 11.82575|
 Finland| 5.604791666666667|
 Uzbekistan| 5.590000000000002|
 Denmark| 5.3412500000000005|
 Kazakhstan| 5.3134999999999994|
 Mongolia| 5.282833333333333|
 Canada| 5.254000000000001|
United Arab Emirates| 5.143333333333334|
 Ukraine| 5.102333333333331|
 Russia| 4.739625|
 Estonia| 4.723166666666667|
 Azerbaijan| 4.688375000000002|
 Romania| 4.570833333333334|
 Belarus| 4.570333333333336|
 Latvia| 4.536083333333334|
 Georgia| 4.517666666666664|
 Armenia| 4.514916666666669|
 Hungary| 4.513666666666668|
 Bulgaria| 4.456291666666667|
 Sweden| 4.406333333333333|
+--------------------+----------------------+
only showing top 20 rows

We want the First World Countries (top five) have had the biggest change in average land temperature between 1850 and 2013.

Filter the first five rows where diffAverageTemperature > 5.3.

(We could use .show(5), however the code given enables us to display the data in the next section)

In [0]:
# Filter first five rows of diff_fw_country_land_temp_sdf
diff_fw_country_land_temp_sdf = diff_fw_country_land_temp_sdf.filter(col('diffAverageTemperature') > 5.3)

diff_fw_country_land_temp_sdf.show()

+------------+----------------------+
 Country|diffAverageTemperature|
+------------+----------------------+
Saudi Arabia| 11.82575|
 Finland| 5.604791666666667|
 Uzbekistan| 5.590000000000002|
 Denmark| 5.3412500000000005|
 Kazakhstan| 5.3134999999999994|
+------------+----------------------+

###Stage 4: Data Visualisation

For the final stage, we display the data that shows the top five First World Countries that have had the biggest change in average land temperature between 1850 and 2013.

In [0]:
# Visualise the data in diff_fw_country_land_temp_sdf
display(diff_fw_country_land_temp_sdf)

Country,diffAverageTemperature
Saudi Arabia,11.82575
Finland,5.604791666666667
Uzbekistan,5.590000000000002
Denmark,5.3412500000000005
Kazakhstan,5.3134999999999994


Therefore in descending order, Saudi Arabia, Finland, Uzbekistan, Denmark and Kazakhstan have had the biggest change in average land temperature, i.e. 11.83°C, 5.60°C, 5.59°C, 5.34°C and 5.31°C respectively, between 1850 and 2013.

##Goal 2: Top five Cities of the five First World Countries from Goal 1 that have seen the biggest change in average land temperature between 1850 and 2013

In this section we aim to discover, using the results found in Goal 1 and the Global Land Temperatures By City dataset, the top five cities of the First World Countries Saudi Arabia, Finland, Uzbekistan, Denmark and Kazakhstan that have seen the biggest change in average land temperature between 1850 and 2013. To achieve this aim, we will use the data analytic pipeline described at the beginning of this notebook.

###Stage 1: Data Collection

Read in the Global Land Temeratures By City dataset as a Spark DataFrame.

In [0]:
# Read Global Land Temeratures By City dataset from  /FileStore/tables/GlobalLandTemperaturesByCity.csv

city_land_temp_sdf = (sqlContext.read.format("csv").
  option("header", "true").
  option("delimiter", ",").                      
  option("nullValue", "NA").                      
  option("inferSchema", True).
  load("/FileStore/tables/GlobalLandTemperaturesByCity.csv"))

city_land_temp_sdf.printSchema()

root
-- dt: timestamp (nullable = true)
-- AverageTemperature: double (nullable = true)
-- AverageTemperatureUncertainty: double (nullable = true)
-- City: string (nullable = true)
-- Country: string (nullable = true)
-- Latitude: string (nullable = true)
-- Longitude: string (nullable = true)

In [0]:
# See Global Land Temeratures By Major City data
display(city_land_temp_sdf)

dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
1743-11-01T00:00:00.000+0000,6.068,1.7369999999999999,Århus,Denmark,57.05N,10.33E
1743-12-01T00:00:00.000+0000,null,null,Århus,Denmark,57.05N,10.33E
1744-01-01T00:00:00.000+0000,null,null,Århus,Denmark,57.05N,10.33E
1744-02-01T00:00:00.000+0000,null,null,Århus,Denmark,57.05N,10.33E
1744-03-01T00:00:00.000+0000,null,null,Århus,Denmark,57.05N,10.33E
1744-04-01T00:00:00.000+0000,5.7879999999999985,3.6239999999999997,Århus,Denmark,57.05N,10.33E
1744-05-01T00:00:00.000+0000,10.644,1.2830000000000001,Århus,Denmark,57.05N,10.33E
1744-06-01T00:00:00.000+0000,14.050999999999998,1.347,Århus,Denmark,57.05N,10.33E
1744-07-01T00:00:00.000+0000,16.082,1.396,Århus,Denmark,57.05N,10.33E
1744-08-01T00:00:00.000+0000,null,null,Århus,Denmark,57.05N,10.33E


###Stage 2: Data Transformation

Taking city_land_temp_sdf, extract year and month from dt and select the required features.

In [0]:
# Extract year and month from dt and select features

city_land_temp_sdf_cleaned = city_land_temp_sdf.select(year(city_land_temp_sdf['dt']).alias('year'),
                                                       month(city_land_temp_sdf['dt']).alias('month'),
                                                       city_land_temp_sdf['AverageTemperature'],
                                                       city_land_temp_sdf['City'],      
                                                       city_land_temp_sdf['Country'])

city_land_temp_sdf_cleaned.show()

+----+-----+-------------------+-----+-------+
year|month| AverageTemperature| City|Country|
+----+-----+-------------------+-----+-------+
1743| 11| 6.068|Århus|Denmark|
1743| 12| null|Århus|Denmark|
1744| 1| null|Århus|Denmark|
1744| 2| null|Århus|Denmark|
1744| 3| null|Århus|Denmark|
1744| 4| 5.7879999999999985|Århus|Denmark|
1744| 5| 10.644|Århus|Denmark|
1744| 6| 14.050999999999998|Århus|Denmark|
1744| 7| 16.082|Århus|Denmark|
1744| 8| null|Århus|Denmark|
1744| 9| 12.780999999999999|Århus|Denmark|
1744| 10| 7.95|Århus|Denmark|
1744| 11| 4.638999999999999|Århus|Denmark|
1744| 12|0.12199999999999987|Århus|Denmark|
1745| 1|-1.3330000000000002|Århus|Denmark|
1745| 2| -2.732|Århus|Denmark|
1745| 3| 0.129|Århus|Denmark|
1745| 4| 4.042|Århus|Denmark|
1745| 5| null|Århus|Denmark|
1745| 6| null|Århus|Denmark|
+----+-----+-------------------+-----+-------+
only showing top 20 rows

We want years 1850-2013 and the cities of the First World Countries Saudi Arabia, Finland, Uzbekistan, Denmark and Kazakhstan.
Filter city_land_temp_sdf_cleaned for years greater than 1849 and then filter for the countries mentioned.

In [0]:
# Filter year > 1849 and countries Saudi Arabia, Finland, Uzbekistan, Denmark and Kazakhstan
city_land_temp_sdf_cleaned = city_land_temp_sdf_cleaned.filter(city_land_temp_sdf_cleaned['year'] > 1849)

city_land_temp_sdf_cleaned = city_land_temp_sdf_cleaned.filter((col('Country') == 'Saudi Arabia') | (col('Country') == 'Finland') | (col('Country') == 'Uzbekistan') | (col('Country') == 'Denmark') | (col('Country') == 'Kazakhstan'))

city_land_temp_sdf_cleaned.show()

+----+-----+--------------------+-----+-------+
year|month| AverageTemperature| City|Country|
+----+-----+--------------------+-----+-------+
1850| 1| -5.265|Århus|Denmark|
1850| 2| 1.859|Århus|Denmark|
1850| 3|0.031999999999999806|Århus|Denmark|
1850| 4| 5.7639999999999985|Århus|Denmark|
1850| 5| 11.037|Århus|Denmark|
1850| 6| 15.299|Århus|Denmark|
1850| 7| 16.832|Århus|Denmark|
1850| 8| 16.07|Århus|Denmark|
1850| 9| 11.837|Århus|Denmark|
1850| 10| 6.424|Århus|Denmark|
1850| 11| 3.2739999999999996|Århus|Denmark|
1850| 12| 1.743|Århus|Denmark|
1851| 1| 0.9640000000000001|Århus|Denmark|
1851| 2| 0.992|Århus|Denmark|
1851| 3| 1.301|Århus|Denmark|
1851| 4| 6.1869999999999985|Århus|Denmark|
1851| 5| 8.975|Århus|Denmark|
1851| 6| 13.494000000000002|Århus|Denmark|
1851| 7| 15.352|Århus|Denmark|
1851| 8| 15.550999999999998|Århus|Denmark|
+----+-----+--------------------+-----+-------+
only showing top 20 rows

Drop any null values from city_land_temp_sdf_cleaned.

In [0]:
# Drop nulls
city_land_temp_sdf_cleaned = city_land_temp_sdf_cleaned.dropna()

city_land_temp_sdf_cleaned.show()

+----+-----+--------------------+-----+-------+
year|month| AverageTemperature| City|Country|
+----+-----+--------------------+-----+-------+
1850| 1| -5.265|Århus|Denmark|
1850| 2| 1.859|Århus|Denmark|
1850| 3|0.031999999999999806|Århus|Denmark|
1850| 4| 5.7639999999999985|Århus|Denmark|
1850| 5| 11.037|Århus|Denmark|
1850| 6| 15.299|Århus|Denmark|
1850| 7| 16.832|Århus|Denmark|
1850| 8| 16.07|Århus|Denmark|
1850| 9| 11.837|Århus|Denmark|
1850| 10| 6.424|Århus|Denmark|
1850| 11| 3.2739999999999996|Århus|Denmark|
1850| 12| 1.743|Århus|Denmark|
1851| 1| 0.9640000000000001|Århus|Denmark|
1851| 2| 0.992|Århus|Denmark|
1851| 3| 1.301|Århus|Denmark|
1851| 4| 6.1869999999999985|Århus|Denmark|
1851| 5| 8.975|Århus|Denmark|
1851| 6| 13.494000000000002|Århus|Denmark|
1851| 7| 15.352|Århus|Denmark|
1851| 8| 15.550999999999998|Århus|Denmark|
+----+-----+--------------------+-----+-------+
only showing top 20 rows

###Stage 3: Descriptive Analysis

Taking city_land_temp_sdf_cleaned, calculate the yearly average temperature for each city and project to a new DataFrame city_yearly_land_temp_sdf with features year, City, Country and YearlyAverageTemperature.

In [0]:
# YearlyAverageTemperature for each City
city_yearly_land_temp_sdf = city_land_temp_sdf_cleaned.groupBy('year', 'City', 'Country').agg(avg(col('AverageTemperature')).alias('YearlyAverageTemperature')).orderBy('Country', 'City', 'year')

city_yearly_land_temp_sdf.show()

+----+-------+-------+------------------------+
year| City|Country|YearlyAverageTemperature|
+----+-------+-------+------------------------+
1850|Aalborg|Denmark| 7.075500000000001|
1851|Aalborg|Denmark| 7.520083333333333|
1852|Aalborg|Denmark| 8.046666666666667|
1853|Aalborg|Denmark| 6.796916666666667|
1854|Aalborg|Denmark| 7.750583333333334|
1855|Aalborg|Denmark| 6.1284166666666655|
1856|Aalborg|Denmark| 6.761750000000002|
1857|Aalborg|Denmark| 8.356416666666666|
1858|Aalborg|Denmark| 7.809916666666667|
1859|Aalborg|Denmark| 8.449833333333332|
1860|Aalborg|Denmark| 6.576583333333333|
1861|Aalborg|Denmark| 7.691083333333332|
1862|Aalborg|Denmark| 7.2210833333333335|
1863|Aalborg|Denmark| 8.568416666666666|
1864|Aalborg|Denmark| 6.636749999999999|
1865|Aalborg|Denmark| 7.469333333333334|
1866|Aalborg|Denmark| 7.80275|
1867|Aalborg|Denmark| 6.34625|
1868|Aalborg|Denmark| 8.668083333333334|
1869|Aalborg|Denmark| 7.573333333333334|
+----+-------+-------+------------------------+
only showing top 20 rows

Taking city_yearly_land_temp_sdf, calculate the max and min average temperature for each city and project each computation to new DataFrames max_city_land_temp_sdf and min_city_land_temp_sdf. Then merge these new DataFrames and call this maxmin_city_land_temp_sdf.

In [0]:
# max_city_land_temp_sdf

max_city_land_temp_sdf = city_yearly_land_temp_sdf.groupBy('City', 'Country').agg(max(col('YearlyAverageTemperature')).alias('MaxAverageTemperature')).orderBy('Country', 'City')

max_city_land_temp_sdf.createOrReplaceTempView("max_city_land_temp_sdf")

max_city_land_temp_sdf.show()

+----------+----------+---------------------+
 City| Country|MaxAverageTemperature|
+----------+----------+---------------------+
 Aalborg| Denmark| 9.673833333333333|
Copenhagen| Denmark| 9.87325|
 Odense| Denmark| 9.87325|
 Århus| Denmark| 9.673833333333333|
 Espoo| Finland| 6.767833333333333|
 Helsinki| Finland| 6.767833333333333|
 Oulu| Finland| 4.27475|
 Tampere| Finland| 6.057749999999999|
 Turku| Finland| 7.2610833333333344|
 Aktau|Kazakhstan| 15.09875|
 Almaty|Kazakhstan| 8.7145|
 Aqtöbe|Kazakhstan| 7.762625|
 Astana|Kazakhstan| 5.491375|
 Atyrau|Kazakhstan| 12.658625|
 Ekibastuz|Kazakhstan| 5.457249999999998|
 Pavlodar|Kazakhstan| 5.457249999999998|
 Petropavl|Kazakhstan| 3.7459166666666674|
 Qostanay|Kazakhstan| 4.528083333333334|
 Semey|Kazakhstan| 6.283625000000001|
 Shymkent|Kazakhstan| 14.561999999999998|
+----------+----------+---------------------+
only showing top 20 rows

In [0]:
# min_city_land_temp_sdf

min_city_land_temp_sdf = city_yearly_land_temp_sdf.groupBy('City', 'Country').agg(min(col('YearlyAverageTemperature')).alias('MinAverageTemperature')).orderBy('Country', 'City').\
                                select(col('City').alias('City_x'), col('Country').alias('Country_x'), 'MinAverageTemperature')

min_city_land_temp_sdf.createOrReplaceTempView("min_city_land_temp_sdf")

min_city_land_temp_sdf.show()

+----------+----------+---------------------+
 City_x| Country_x|MinAverageTemperature|
+----------+----------+---------------------+
 Aalborg| Denmark| 6.050833333333333|
Copenhagen| Denmark| 6.1935|
 Odense| Denmark| 6.1935|
 Århus| Denmark| 6.050833333333333|
 Espoo| Finland| 1.7174999999999996|
 Helsinki| Finland| 1.7174999999999996|
 Oulu| Finland| -1.5105833333333338|
 Tampere| Finland| 1.0246666666666668|
 Turku| Finland| 2.2956666666666665|
 Aktau|Kazakhstan| 9.223333333333334|
 Almaty|Kazakhstan| 3.827166666666667|
 Aqtöbe|Kazakhstan| 1.4780833333333323|
 Astana|Kazakhstan| 0.19516666666666657|
 Atyrau|Kazakhstan| 6.136|
 Ekibastuz|Kazakhstan| -0.07700000000000055|
 Pavlodar|Kazakhstan| -0.07700000000000055|
 Petropavl|Kazakhstan| -1.3891666666666669|
 Qostanay|Kazakhstan| -0.4383333333333339|
 Semey|Kazakhstan| 0.325083333333333|
 Shymkent|Kazakhstan| 9.026916666666668|
+----------+----------+---------------------+
only showing top 20 rows

In [0]:
# Merge
maxmin_city_land_temp_sdf = sqlContext.sql("select City, Country, maxAverageTemperature, minAverageTemperature FROM max_city_land_temp_sdf join min_city_land_temp_sdf ON City=City_x ORDER BY Country, City")

maxmin_city_land_temp_sdf.show()

+----------+----------+---------------------+---------------------+
 City| Country|maxAverageTemperature|minAverageTemperature|
+----------+----------+---------------------+---------------------+
 Aalborg| Denmark| 9.673833333333333| 6.050833333333333|
Copenhagen| Denmark| 9.87325| 6.1935|
 Odense| Denmark| 9.87325| 6.1935|
 Århus| Denmark| 9.673833333333333| 6.050833333333333|
 Espoo| Finland| 6.767833333333333| 1.7174999999999996|
 Helsinki| Finland| 6.767833333333333| 1.7174999999999996|
 Oulu| Finland| 4.27475| -1.5105833333333338|
 Tampere| Finland| 6.057749999999999| 1.0246666666666668|
 Turku| Finland| 7.2610833333333344| 2.2956666666666665|
 Aktau|Kazakhstan| 15.09875| 9.223333333333334|
 Almaty|Kazakhstan| 8.7145| 3.827166666666667|
 Aqtöbe|Kazakhstan| 7.762625| 1.4780833333333323|
 Astana|Kazakhstan| 5.491375| 0.19516666666666657|
 Atyrau|Kazakhstan| 12.658625| 6.136|
 Ekibastuz|Kazakhstan| 5.457249999999998| -0.07700000000000055|
 Pavlodar|Kazakhstan| 5.457249999999998| -0.07700000000000055|
 Petropavl|Kazakhstan| 3.7459166666666674| -1.3891666666666669|
 Qostanay|Kazakhstan| 4.528083333333334| -0.4383333333333339|
 Semey|Kazakhstan| 6.283625000000001| 0.325083333333333|
 Shymkent|Kazakhstan| 14.561999999999998| 9.026916666666668|
+----------+----------+---------------------+---------------------+
only showing top 20 rows

Taking maxmin_city_land_temp_sdf, calculate the difference of max and min average temperature for each city and project to a new DataFrame diff_city_land_temp_sdf. We use the absolute value function to ensure the values are positive.

Additionally, as we want the cities associated to the First World Countries Saudi Arabia, Finland, Uzbekistan, Denmark and Kazakhstan that have seen the biggest change in average land temperature between 1850 and 2013, we order the feature diffAverageTemperature in descending order.

In [0]:
# diff_city_land_temp_sdf

diff_city_land_temp_sdf = maxmin_city_land_temp_sdf.select('City', 'Country',
                                                           abs(col('maxAverageTemperature') - col('minAverageTemperature')).alias('diffAverageTemperature')).orderBy('diffAverageTemperature', ascending=False)

diff_city_land_temp_sdf.show()

+----------+----------+----------------------+
 City| Country|diffAverageTemperature|
+----------+----------+----------------------+
 Atyrau|Kazakhstan| 6.522625000000001|
 Nukus|Uzbekistan| 6.355083333333335|
 Aqtöbe|Kazakhstan| 6.284541666666668|
 Semey|Kazakhstan| 5.958541666666668|
 Aktau|Kazakhstan| 5.875416666666666|
 Oulu| Finland| 5.785333333333334|
 Öskemen|Kazakhstan| 5.584500000000001|
 Shymkent|Kazakhstan| 5.535083333333329|
 Ekibastuz|Kazakhstan| 5.534249999999999|
 Pavlodar|Kazakhstan| 5.534249999999999|
Zhezkazgan|Kazakhstan| 5.489625|
 Astana|Kazakhstan| 5.296208333333333|
 Temirtau|Kazakhstan| 5.296208333333333|
 Angren|Uzbekistan| 5.248708333333337|
 Tashkent|Uzbekistan| 5.248708333333337|
 Navoi|Uzbekistan| 5.235000000000001|
 Petropavl|Kazakhstan| 5.135083333333334|
 Taraz|Kazakhstan| 5.1305|
 Helsinki| Finland| 5.050333333333334|
 Espoo| Finland| 5.050333333333334|
+----------+----------+----------------------+
only showing top 20 rows

We want the cities (top five) associated to the First World Countries Saudi Arabia, Finland, Uzbekistan, Denmark and Kazakhstan that have seen the biggest change in average land temperature between 1850 and 2013.

Filter the first five rows where diffAverageTemperature > 5.8.

(We could use .show(5), however the code given enables us to display the data in the next section)

In [0]:
# Filter first five rows of diff_city_land_temp_sdf
diff_city_land_temp_sdf = diff_city_land_temp_sdf.filter(col('diffAverageTemperature') > 5.8)

diff_city_land_temp_sdf.show()

+------+----------+----------------------+
 City| Country|diffAverageTemperature|
+------+----------+----------------------+
Atyrau|Kazakhstan| 6.522625000000001|
 Nukus|Uzbekistan| 6.355083333333335|
Aqtöbe|Kazakhstan| 6.284541666666668|
 Semey|Kazakhstan| 5.958541666666668|
 Aktau|Kazakhstan| 5.875416666666666|
+------+----------+----------------------+

###Stage 4: Data Visualisation

For the final stage, we display the data that shows the top five cities associated to the First World Countries Saudi Arabia, Finland, Uzbekistan, Denmark and Kazakhstan that have seen the biggest change in average land temperature between 1850 and 2013.

In [0]:
# Visualise the data in diff_city_land_temp_sdf
display(diff_city_land_temp_sdf)

City,Country,diffAverageTemperature
Atyrau,Kazakhstan,6.522625000000001
Nukus,Uzbekistan,6.355083333333335
Aqtöbe,Kazakhstan,6.284541666666668
Semey,Kazakhstan,5.958541666666668
Aktau,Kazakhstan,5.875416666666666


Therefore in descending order, the cities Atyrau, Nukus, Aqtöbe, Semey and Aktau have seen the biggest change in average land temperature, i.e. 6.52°C, 6.36°C, 6.28°C, 5.96°C and 5.88°C respectively, between 1850 and 2013.

##Goal 3: Predict the average land temperature of the City found to have the greatest change in average land temperature from Goal 2 for the years 2014-2020.

In this section we aim to predict, using the results found in Goal 2 and the Global Land Temperatures By City dataset, the average land temperature of the City found to have the greatest change in average temperature, i.e. Atyrau in Kazakhstan, for the years 2014-2020. To achieve this aim, we will use the data analytic pipeline described at the beginning of this notebook.

###Stage 1: Data Collection

From Goal 2, we were able to transform the Global Land Temperature By City dataset to the DataFrame city_yearly_land_temp_sdf. This DataFrame is a good starting point for Goal 3.

In [0]:
# See city_yearly_land_temp_sdf
display(city_yearly_land_temp_sdf)

year,City,Country,YearlyAverageTemperature
1850,Aalborg,Denmark,7.075500000000001
1851,Aalborg,Denmark,7.520083333333333
1852,Aalborg,Denmark,8.046666666666667
1853,Aalborg,Denmark,6.796916666666667
1854,Aalborg,Denmark,7.750583333333334
1855,Aalborg,Denmark,6.1284166666666655
1856,Aalborg,Denmark,6.761750000000002
1857,Aalborg,Denmark,8.356416666666666
1858,Aalborg,Denmark,7.809916666666667
1859,Aalborg,Denmark,8.449833333333332


###Stage 2: Data Transformation

As we are only interested in predicting the average land temperature of the city Atyrau for the years 2014-2020, we filter the city_yearly_land_temp_sdf for this city.

In [0]:
# Filter rows in city_yearly_land_temp_sdf for the city Atyrau
atyrau_yearly_land_temp_sdf = city_yearly_land_temp_sdf.filter(city_yearly_land_temp_sdf['City'] == 'Atyrau')

atyrau_yearly_land_temp_sdf.show()

+----+------+----------+------------------------+
year| City| Country|YearlyAverageTemperature|
+----+------+----------+------------------------+
1850|Atyrau|Kazakhstan| 7.94025|
1851|Atyrau|Kazakhstan| 8.536416666666666|
1852|Atyrau|Kazakhstan| 7.601500000000001|
1853|Atyrau|Kazakhstan| 7.849833333333334|
1854|Atyrau|Kazakhstan| 8.053166666666668|
1855|Atyrau|Kazakhstan| 8.794583333333334|
1856|Atyrau|Kazakhstan| 8.18525|
1857|Atyrau|Kazakhstan| 7.010000000000002|
1858|Atyrau|Kazakhstan| 7.354083333333333|
1859|Atyrau|Kazakhstan| 8.622583333333333|
1860|Atyrau|Kazakhstan| 7.151749999999999|
1861|Atyrau|Kazakhstan| 6.714666666666666|
1862|Atyrau|Kazakhstan| 6.147916666666667|
1863|Atyrau|Kazakhstan| 8.386750000000001|
1864|Atyrau|Kazakhstan| 7.4789166666666675|
1865|Atyrau|Kazakhstan| 7.408500000000001|
1866|Atyrau|Kazakhstan| 7.9623333333333335|
1867|Atyrau|Kazakhstan| 7.935000000000001|
1868|Atyrau|Kazakhstan| 7.295333333333333|
1869|Atyrau|Kazakhstan| 8.395416666666666|
+----+------+----------+------------------------+
only showing top 20 rows

Our feature is year and our label is YearlyAverageTemperature.

In [0]:
atyrau_yearly_land_temp_sdf = atyrau_yearly_land_temp_sdf.select('year', 'YearlyAverageTemperature')

atyrau_yearly_land_temp_sdf.show()

+----+------------------------+
year|YearlyAverageTemperature|
+----+------------------------+
1850| 7.94025|
1851| 8.536416666666666|
1852| 7.601500000000001|
1853| 7.849833333333334|
1854| 8.053166666666668|
1855| 8.794583333333334|
1856| 8.18525|
1857| 7.010000000000002|
1858| 7.354083333333333|
1859| 8.622583333333333|
1860| 7.151749999999999|
1861| 6.714666666666666|
1862| 6.147916666666667|
1863| 8.386750000000001|
1864| 7.4789166666666675|
1865| 7.408500000000001|
1866| 7.9623333333333335|
1867| 7.935000000000001|
1868| 7.295333333333333|
1869| 8.395416666666666|
+----+------------------------+
only showing top 20 rows

###Stage 3: Feature Engineering

The feature column year must be wrapped as vector value.

In [0]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ["year"], outputCol = "features")

new_atyrau_yearly_land_temp_sdf = assembler.transform(atyrau_yearly_land_temp_sdf)

###Stage 4: Algorithm Selection

The algorithms that will be used to achieve Goal 3 are the Supervised Machine Learning Algorithms Linear Regression, Decision Tree Regression and Random Forest Regression from MLlib, Apache Spark's scalable machine learning library (https://spark.apache.org/mllib/).

####Stage 4.1: Defining the Linear Regression Algorithm

In [0]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol="features", labelCol="YearlyAverageTemperature")

####Stage 4.2: Defining the Decision Tree Regression Algorithm

In [0]:
from pyspark.ml.regression import DecisionTreeRegressor

dtr = DecisionTreeRegressor(featuresCol="features", labelCol="YearlyAverageTemperature")

####Stage 4.3: Defining the Random Forest Regression Algorithm

In [0]:
from pyspark.ml.regression import RandomForestRegressor

rfr = RandomForestRegressor(featuresCol="features",labelCol="YearlyAverageTemperature")

###Stage 5: Training

Split data into training data and evaluation data (ratio is 80% : 20%)

In [0]:
# Split data into train and test set
train, test  = new_atyrau_yearly_land_temp_sdf.randomSplit([0.8,0.2])

Next train each of the three algorithms.

####Stage 5.1: Training the Linear Regression Algorithm

In [0]:
# Train Linear Regression algorithm
lr_model = lr.fit(train)

####Stage 5.2: Training the Decision Tree Regression Algorithm

In [0]:
# Train Decision Tree Regression algorithm
dtr_model = dtr.fit(train)

####Stage 5.3: Training the Random Forest Regression Algorithm

In [0]:
# Train Random Forest Regression algorithm
rfr_model = rfr.fit(train)

###Stage 6: Testing

Before testing each of three models, initiate the evaluation metrics Mean Absolute Error (MAE), Root Mean Square Error (RMSE) and R-squared.

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

# Mean Absolute Error Evaluator
evaluator_mae = RegressionEvaluator(metricName="mae", predictionCol="prediction",labelCol="YearlyAverageTemperature")

# Root Mean Square Error Evaluator
evaluator_rmse = RegressionEvaluator(metricName="rmse", predictionCol="prediction",labelCol="YearlyAverageTemperature")

# R-squared Evaluator
evaluator_r2 = RegressionEvaluator(metricName="r2", predictionCol="prediction",labelCol="YearlyAverageTemperature")

Now test and evaluate each of the three models.

####Stage 6.1: Testing the Linear Regression Model

In [0]:
# Test Linear Regression model
test_lr = lr_model.transform(test)

# Print MAE MSE and R2
print("linear regression mae:", evaluator_mae.evaluate(test_lr))
print("linear regression rmse:", evaluator_rmse.evaluate(test_lr))
print("linear regression r2:", evaluator_r2.evaluate(test_lr))

linear regression mae: 0.7396496465646434
linear regression rmse: 0.913781671762892
linear regression r2: 0.23936627012115252

####Stage 6.2: Testing the Decision Tree Regression Model

In [0]:
# Test Decision Tree Regression model
test_dtr = dtr_model.transform(test)

# Print MAE MSE and R2
print("decision tree regressor mae:", evaluator_mae.evaluate(test_dtr))
print("decision tree regressor rmse:", evaluator_rmse.evaluate(test_dtr))
print("decision tree regressor r2:", evaluator_r2.evaluate(test_dtr))

decision tree regressor mae: 0.6773427656971089
decision tree regressor rmse: 0.869902424275872
decision tree regressor r2: 0.31066270358066783

####Stage 6.3: Testing the Random Forest Regression Model

In [0]:
# Test Random Forest Regression model
test_rfr = rfr_model.transform(test)

# Print MAE MSE and R2
print("random forests regressor mae:", evaluator_mae.evaluate(test_rfr))
print("random forests regressor rmse:", evaluator_rmse.evaluate(test_rfr))
print("random forests regressor r2:", evaluator_r2.evaluate(test_rfr))

random forests regressor mae: 0.6721588656930307
random forests regressor rmse: 0.8687371342246318
random forests regressor r2: 0.3125082895911291

###Stage 7: Predictive Analysis

In this stage of the pipeline, we take each of our three models and predict the average land temperature of the city Atyrau for the years 2014-2020.

First we create a DataFrame (series) with the feature year with values 2014, 2015, 2016, 2017, 2018, 2019 and 2020.

In [0]:
# prepare data
year = [2014, 2015, 2016, 2017, 2018, 2019, 2020] 
l = list(zip(year))
l

Out[48]: [(2014,), (2015,), (2016,), (2017,), (2018,), (2019,), (2020,)]

In [0]:
# pyspark dataframe
from pyspark.sql import Row

rdd = sc.parallelize(l)
rows = rdd.map(lambda z: Row(year=int(z[0])))
pred_sdf = spark.createDataFrame(rows)

# display data
display(pred_sdf)

# Feature Engineering: vector assembler
va_pred_sdf = assembler.transform(pred_sdf)

year
2014
2015
2016
2017
2018
2019
2020


Make predictions based on each of the three models.

####Stage 7.1: Predictions based on the Linear Regression Model

In [0]:
lr_pred = lr_model.transform(va_pred_sdf)
lr_pred_res = lr_pred.select("year", "prediction")
lr_pred_res.cache()

Out[50]: DataFrame[year: bigint, prediction: double]

####Stage 7.2: Predictions based on the Decision Tree Regression Model

In [0]:
dtr_pred = dtr_model.transform(va_pred_sdf)
dtr_pred_res = dtr_pred.select("year", "prediction")
dtr_pred_res.cache()

Out[51]: DataFrame[year: bigint, prediction: double]

####Stage 7.3: Predictions based on the Random Forest Regression Model

In [0]:
rfr_pred = rfr_model.transform(va_pred_sdf)
rfr_pred_res = rfr_pred.select("year", "prediction")
rfr_pred_res.cache()

Out[52]: DataFrame[year: bigint, prediction: double]

###Stage 8: Data Visualisation

Lets see the predictions!

####Stage 8.1: Visualisation of Predictions based on the Linear Regression Model

In [0]:
display(lr_pred_res)

year,prediction
2014,9.32451463157836
2015,9.33732976786327
2016,9.35014490414818
2017,9.362960040433094
2018,9.375775176718005
2019,9.388590313002915
2020,9.401405449287825


####Stage 8.2: Visualisation of Predictions based on the Decision Tree Regression Model

In [0]:
display(dtr_pred_res)

year,prediction
2014,10.148093750000001
2015,10.148093750000001
2016,10.148093750000001
2017,10.148093750000001
2018,10.148093750000001
2019,10.148093750000001
2020,10.148093750000001


####Stage 8.3: Visualisation of Predictions based on the Random Forest Regression Model

In [0]:
display(rfr_pred_res)

year,prediction
2014,10.060988422619051
2015,10.060988422619051
2016,10.060988422619051
2017,10.060988422619051
2018,10.060988422619051
2019,10.060988422619051
2020,10.060988422619051
